# ERDDAP to HighCharts Translator

- Can be run on preliminary or final data
- creates measurement type groups (temp, sal, oxy, chlor)

These should be explorable from a deployment map (and maybe the log sheets).  The goal is to get a dynamic view of the data from the instruments at a site.

Other ways to show information is instrument by instrument (this is done in the QC STAGE to evaluate corresponding issues - like sal/temp spikes)

In [1]:
import pandas as pd
import EcoFOCIpy.io.erddap as erddapy
from erddapy import ERDDAP

In [2]:
status_dic = {'final':{'url':"http://ecofoci-field.pmel.noaa.gov:8080/erddap",'filename':'.csv','dataset':'datasets_Mooring_22bs2c_final'},
          'prelim':{'url':"http://ecofoci-field.pmel.noaa.gov:8082/erddap",'filename':'_prelim.csv','dataset':'datasets_Mooring_22bs2c_preliminary'}}

### Looped Code to do all moorings (first final and then prelim)

In [3]:
dataset_tag = 'datasets_Mooring_24bs'

In [4]:
### do all moorings

status = 'final'
e = ERDDAP(server=status_dic[status]['url'])
df_ds = pd.read_csv(e.get_search_url(response='csv', search_for=f'final {dataset_tag}'))

for dataset_id in df_ds['Dataset ID']:
    print(dataset_id)
    e = ERDDAP(server=status_dic[status]['url'],
              protocol='tabledap',
              response='csv')

    e.dataset_id=dataset_id
    
    pdf = e.to_pandas(
                index_col='time (UTC)',
                parse_dates=True,
                skiprows=(1,),  # units information can be dropped.
                low_memory=False
            )
    pdf.columns = [x.split(' ')[0] for x in pdf.columns]
    pdf.index.name = 'time'
    
    datalabel = dataset_id.split('_')[2]
    # for parameter in ['temperature','salinity','chlorophyll_fluorescence']:
    for parameter in ['temperature','salinity','chlor_fluorescence','chlorophyll_fluorescence','oxygen_concentration']:
        try:
            xdf = pdf[[parameter,'timeseries_id']].pivot_table(index='time', columns=['timeseries_id']).resample('1h').median().interpolate(limit=3).xs(parameter, axis=1, drop_level=True)
            xdf.columns.rename("",inplace=True)
            xdf.index = xdf.index.tz_localize(None)
            xdf.round(3).to_csv(f'{datalabel}_{parameter}.csv')
        except:
            pass

datasets_Mooring_24bs2c_final
datasets_Mooring_24bs4a_final
datasets_Mooring_24bs5a_final
datasets_Mooring_24bs8a_final
datasets_Mooring_24bsp14a_final
datasets_Mooring_24bspr2a_final
datasets_Mooring_24bsitaer8a_final


In [6]:
### do all moorings in prelim

status = 'prelim'
e = ERDDAP(server=status_dic[status]['url'])
df_ds = pd.read_csv(e.get_search_url(response='csv', search_for=f'preliminary {dataset_tag}'))

for dataset_id in df_ds['Dataset ID']:
    if not dataset_id in ['']:
        print(dataset_id)
        e = ERDDAP(server=status_dic[status]['url'],
                  protocol='tabledap',
                  response='csv')
    
        e.dataset_id=dataset_id
    
        pdf = e.to_pandas(
                    index_col='time (UTC)',
                    parse_dates=True,
                    skiprows=(1,),  # units information can be dropped.
                    low_memory=False  
                )
        pdf.columns = [x.split(' ')[0] for x in pdf.columns]
        pdf.index.name = 'time'
        
        datalabel = dataset_id.split('_')[2]
        # for parameter in ['temperature','salinity','chlorophyll_fluorescence']:
        for parameter in ['temperature','salinity','chlor_fluorescence','chlorophyll_fluorescence','oxygen_concentration']:
            try:
                xdf = pdf[[parameter,'timeseries_id']].pivot_table(index='time', columns=['timeseries_id']).resample('1h').median().interpolate(limit=3).xs(parameter, axis=1, drop_level=True)
                xdf.columns.rename("",inplace=True)
                xdf.index = xdf.index.tz_localize(None)
                xdf.round(3).to_csv(f'{datalabel}_{parameter}_prelim.csv')
            except:
                pass

HTTPError: HTTP Error 404: 